# Final Challenge Code:Nation Data Science
***

> ## Purpose:
The purpose of this product is to provide an automated service that recommends leads to a user given their current customer list (Portfolio).

***
## Description of steps:
- Import Libraries and files
- T1 - preparation 1: Collection in the market of missing information in the portfolio;
- T2 - preparation 2: Exclusion of features with high percentage of NA's values and filling of the remaining NA's;
- T3 - preparation 3: Conversion of values to numeric type;
- T4 - preparation 4: Label customers and not customers;
- T5 - preparation 5: Balance dataset and Split train and test data;
- Organization of Datasets preparation for the models;
- Models;
- Benchmarking;
- Apply the best model;
- Results.

***
# Recommender systems

***
## Libraries and Files:

> ### Packages:

In [1]:
# Data handling:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
# models: 
import sklearn
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
# Merics:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# Graphics
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.pylabtools import figsize
%matplotlib inline

> ### Data:

In [2]:
market = pd.read_csv("estaticos_market.csv")

In [3]:
portfolio = pd.read_csv("estaticos_portfolio2.csv")

In [4]:
dicionario = pd.read_csv("dicionario.csv")

In [5]:
# Adjust for viewing:
#pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', len(market.columns))
pd.set_option('display.max_rows', len(market))

import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

***
## T1 - Dataset Supplement Portfolio:
The portfolio dataset has only the column 'id', we will complete with the other data that are present in the dataset market.

In [6]:
# DataFrame handling  Step 1:
market_T1= market.copy()

# Creation of id_num column to help manipulation:
market_T1['id_num'] = market_T1['Unnamed: 0']

# Delete column 'Unnamed: 0':
market_T1 = market_T1.drop(['Unnamed: 0'], axis = 1)

# Complete portfolio generation:
portfolio_T1 = pd.DataFrame(portfolio['id'])
portfolio_T1 = pd.merge(portfolio_T1, market_T1, on='id', how='left')

> ### Heads:

In [7]:
market_T1.head(3)

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,fl_optante_simples,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,vl_potenc_cons_oleo_gas,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,vl_frota,empsetorcensitariofaixarendapopulacao,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_st_suspensa,qt_socios_masculino,qt_socios_feminino,qt_socios_pep,qt_alteracao_socio_total,qt_alteracao_socio_90d,qt_alteracao_socio_180d,qt_alteracao_socio_365d,qt_socios_pj_ativos,qt_socios_pj_nulos,qt_socios_pj_baixados,qt_socios_pj_suspensos,qt_socios_pj_inaptos,vl_idade_media_socios_pj,vl_idade_maxima_socios_pj,vl_idade_minima_socios_pj,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,coligada_mais_nova_baixada,coligada_mais_antiga_baixada,idade_media_coligadas_baixadas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,qt_funcionarios_coligados,qt_funcionarios_coligados_gp,media_funcionarios_coligados_gp,max_funcionarios_coligados_gp,min_funcionarios_coligados_gp,vl_folha_coligados,media_vl_folha_coligados,max_vl_folha_coligados,min_vl_folha_coligados,vl_folha_coligados_gp,media_vl_folha_coligados_gp,max_vl_folha_coligados_gp,min_vl_folha_coligados_gp,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,total_filiais_coligados,media_filiais_coligados,max_filiais_coligados,min_filiais_coligados,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_ate_18,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,grau_instrucao_macro_desconhecido,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,id_num
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14.457534,10 a 15,False,False,Fal

In [8]:
portfolio_T1.head(3)

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,fl_optante_simples,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,vl_potenc_cons_oleo_gas,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,vl_frota,empsetorcensitariofaixarendapopulacao,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_st_suspensa,qt_socios_masculino,qt_socios_feminino,qt_socios_pep,qt_alteracao_socio_total,qt_alteracao_socio_90d,qt_alteracao_socio_180d,qt_alteracao_socio_365d,qt_socios_pj_ativos,qt_socios_pj_nulos,qt_socios_pj_baixados,qt_socios_pj_suspensos,qt_socios_pj_inaptos,vl_idade_media_socios_pj,vl_idade_maxima_socios_pj,vl_idade_minima_socios_pj,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,coligada_mais_nova_baixada,coligada_mais_antiga_baixada,idade_media_coligadas_baixadas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,qt_funcionarios_coligados,qt_funcionarios_coligados_gp,media_funcionarios_coligados_gp,max_funcionarios_coligados_gp,min_funcionarios_coligados_gp,vl_folha_coligados,media_vl_folha_coligados,max_vl_folha_coligados,min_vl_folha_coligados,vl_folha_coligados_gp,media_vl_folha_coligados_gp,max_vl_folha_coligados_gp,min_vl_folha_coligados_gp,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,total_filiais_coligados,media_filiais_coligados,max_filiais_coligados,min_filiais_coligados,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_ate_18,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,grau_instrucao_macro_desconhecido,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,id_num
0,09e95c1a84049001d086470a8f320a19b076f955a89122...,True,AUTARQUIA ESTADUAL OU DO DISTRITO FEDERAL,AC,ADMINISTRACAO PUBLICA,"ADMINISTRACAO PUBLICA, DEFESA E SEGURIDADE SOCIAL",SERVIÇO,11.

> ### Shapes(Rows x Columns):

In [9]:
print('Market:', market_T1.shape)
print('Portfolio:', portfolio_T1.shape)

Market: (462298, 182)
Portfolio: (566, 182)


***
## T2 - NA's Handling

> ### NA's per columns:

In [10]:
# NA's percentage check
qt_na_per_col_portfolio_T1 = pd.DataFrame(portfolio_T1.isna().sum().sort_values(ascending=False).reset_index())
qt_na_per_col_portfolio_T1.columns = ['Column', 'qt_NA']
qt_na_per_col_portfolio_T1['qt_not_NA'] = (len(portfolio_T1) - qt_na_per_col_portfolio_T1['qt_NA'])
qt_na_per_col_portfolio_T1['Percentage_of_NA'] = ((qt_na_per_col_portfolio_T1['qt_NA'] * 100) / len(portfolio_T1)).round(2)
qt_na_per_col_portfolio_T1.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181
Column,grau_instrucao_macro_desconhecido,qt_alteracao_socio_total,qt_alteracao_socio_90d,qt_alteracao_socio_180d,qt_alteracao_socio_365d,idade_media_coligadas_baixadas,coligada_mais_nova_baixada,qt_socios_st_suspensa,coligada_mais_antiga_baixada,vl_total_veiculos_antt,vl_total_veiculos_antt_grupo,vl_total_tancagem,vl_total_tancagem_grupo,qt_socios_pep,qt_socios_pj_nulos,qt_socios_pj_suspensos,qt_socios_pj_inaptos,vl_idade_media_socios_pj,vl_idade_maxima_socios_pj,vl_idade_minima_socios_pj,qt_socios_pj_ativos,qt_socios_pj_baixados,vl_potenc_cons_oleo_gas,grau_instrucao_macro_analfabeto,total_filiais_coligados,max_filiais_coligados,media_filiais_coligados,min_filiais_coligados,qt_socios_feminino,idade_ate_18,vl_folha_coligados,media_vl_folha_coligados,max_vl_folha_coligados,min_vl_folha_coligados,vl_folha_coligados_gp,media_vl_folha_coligados_gp,max_vl_folha_coligados_gp,min_vl_folha_coligados_gp,min_funcionarios_coligados_gp,max_funcionarios_coligados_gp,qt_funcionarios_coligados,media_funcionarios_coligados_gp,qt_funcionarios_coligados_gp,qt_art,sum_faturamento_estimado_coligadas,min_faturamento_est_coligados_gp,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,faturamento_est_coligados,qt_coligados_nula,idade_media_coligadas_ativas,qt_coligados_exterior,qt_coligados_sudeste,qt_coligados_centro,qt_coligados_nordeste,qt_coligados_sul,qt_coligados_norte,qt_coligados_epp,qt_coligados_ltda,qt_coligados_me,qt_coligados_sa,qt_coligadas,coligada_mais_antiga_ativa,qt_coligados_suspensa,coligada_mais_nova_ativa,idade_minima_coligadas,idade_maxima_coligadas,idade_media_coligadas,qt_regioes_coligados,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_ufs_coligados,qt_coligados_mei,qt_ramos_coligados,qt_coligados_industria,qt_coligados_atividade_inativo,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_baixo,qt_coligados_ccivil,qt_coligados_serviço,qt_coligados_comercio,qt_coligados_agropecuaria,qt_socios_masculino,qt_socios_st_regular,idade_minima_socios,idade_media_socios,idade_maxima_socios,qt_socios,qt_socios_pj,qt_socios_pf,de_indicador_telefone,vl_frota,idade_de_19_a_23,vl_total_veiculos_leves,vl_total_veiculos_pesados,empsetorcensitariofaixarendapopulacao,nm_micro_regiao,nm_meso_regiao,idade_acima_de_58,grau_instrucao_macro_escolaridade_fundamental,idade_de_54_a_58,idade_de_24_a_28,grau_instrucao_macro_escolaridade_superior,idade_de_49_a_53,idade_de_44_a_48,idade_de_29_a_33,grau_instrucao_macro_escolaridade_media,tx_crescimento_24meses,tx_crescimento_12meses,idade_de_34_a_38,idade_de_39_a_43,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,fl_matriz,fl_optante_simples,fl_epp,de_natureza_juridica,dt_situacao,fl_st_especial,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,fl_ltda,sg_uf,fl_mei,fl_sa,fl_me,idade_emp_cat,idade_empresa_anos,setor,de_ramo,natureza_juridica_macro,fl_email,id_num,fl_optante_simei,min_meses_servicos_all,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,media_meses_servicos,sg_uf_matriz,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_f

> ### Exclusion High Percent NA Columns:

In [11]:
# DataFrame preparation Step 2:
market_T2 = market_T1.copy()

In [12]:
# NA Limit
percent_NA = 55

In [13]:
# Selection of all columns that have high NA's:
column_NA_exclusion = qt_na_per_col_portfolio_T1['Column'].loc[qt_na_per_col_portfolio_T1['Percentage_of_NA'] >= percent_NA]
# Exclusion of columns:
market_T2 = market_T2.drop(column_NA_exclusion, axis = 1)

In [14]:
market_T2.shape

(462298, 139)

> ### Data Types:

In [15]:
# Types:
types_market_T2 = pd.DataFrame(market_T2.dtypes).reset_index()
types_market_T2.columns = ['Columns', 'types']
types_market_T2.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138
Columns,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,vl_total_veiculos_leves,vl_total_veiculos_pesados,fl_optante_simples,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,vl_frota,empsetorcensitariofaixarendapopulacao,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_masculino,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,id_num
types,object,bool,object,object,object,object,object,float64,object,bool,bool,bool,bool,bool,object,bool,bool,bool,object,object,object,object,object,object,float64,float64,object,float64,float64,float64,object,object,object,object,float64,object,object,object,float64,float64,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,float64,float64,float

In [16]:
# type lists:
type_float = list (market_T2.select_dtypes (include = ['float']). columns)
type_int = list (market_T2.select_dtypes (include = ['int']). columns)
type_bool = list (market_T2.select_dtypes (include = ['bool']). columns)
type_object = list (market_T2.select_dtypes (include = ['object']). columns)

In [17]:
# Removal of identifying columns from type lists:
type_object.remove('id')
type_int.remove('id_num')

> ### Fill NA'S:
- Numeric variables by 0;
- Categorical variables by value 'INDETERMINADA';

In [18]:
# Fill Na's
market_T2[type_int] = market_T2[type_int].fillna(0)
market_T2[type_float] = market_T2[type_float].fillna(0)
market_T2[type_object] = market_T2[type_object].fillna('INDETERMINADA')

In [19]:
# Dataframe after preparation:
market_T2.head(3)

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,vl_total_veiculos_leves,vl_total_veiculos_pesados,fl_optante_simples,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,vl_frota,empsetorcensitariofaixarendapopulacao,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_masculino,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,id_num
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,14.457534,10 a 15,False,False,False,False,False,2005-03-25,False,True,True,SIM,CONSTRUCAO DE EDIFICIOS,CONSTRUCAO,False,False,False,0.0,0.0,True,8.0,0.0,0.0,False,RN,VERDE,ACIMA DE 1 ANO,23.0,ALTA,BOA,False,0.0,0.00,LESTE POTIGUAR,NATAL,True,2.0,2.0,0.0,44.0,47.0,41.0,2.0,2.0,6.0,8.0,6.0,6.0,0.0,0.0,0.0,0.0,220.477778,536.366667,57.3,57.3,536.366667,220.477778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,2.0,1.0,5.0,0.0,0.0,1.0,5.0,0.0,21809443.0,3.634907e+06,13188096.0,210000.0,22019443.0,3.669907e+06,13188096.0,210000.0,0.0,0.0,0.0,0.0,0.0,6.0,21809443.0,"DE R$ 1.500.000,01 A R$ 4.800.000,00","DE R$ 1.500.000,01 A R$ 4.800.000,00",3132172.8,3132172.8,119.0,26.0,100.0,0.0,1.0,4.0,6.0,5.0,3.0,4.0,2.0,0.0,1.0,1.0,25.0,0.0,26.0,19.166667,0.0,0.0,119.0,145.0,13.309195,93.266667,-0.933333,43.738462,93.266667,19.166667,26.0,26.0,27.0,0.0,-3.703704,0.0,0,0
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,True,EMPRESARIO INDIVIDUAL,PI,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,1.463014,1 a 5,False,False,False,True,False,2017-05-12,False,True,True,SIM,ALIMENTACAO,

In [20]:
# Comparative:
print('Starter Market:', market_T1.shape)
print('Market after handling:', market_T2.shape)
print('Unimportant columns deleted:', market_T1.shape[1] - market_T2.shape[1])

Starter Market: (462298, 182)
Market after handling: (462298, 139)
Unimportant columns deleted: 43


***
## T3 - Variable coding:

In [21]:
# DataFrame preparation step 5:
market_T3 = market_T2.copy()
market_T3_a = market_T3[['id', 'id_num']]
# Exclusion of identification variables:
market_T3_b = market_T3.drop(['id', 'id_num'], axis = 1)

In [22]:
# Conversions:
market_T3_b[type_float] = market_T3_b[type_float].astype('int')
market_T3_b[type_bool] = market_T3_b[type_bool].astype('category')
market_T3_b[type_bool] = market_T3_b[type_bool].apply(lambda x: x.cat.codes)
market_T3_b[type_object] = market_T3_b[type_object].astype('category')
market_T3_b[type_object] = market_T3_b[type_object].apply(lambda x: x.cat.codes)

In [23]:
# Market Reconstruction:
market_T3 = pd.DataFrame()
market_T3 = pd.concat([market_T3_a, market_T3_b], axis=1)

> ***

In [24]:
# Market after T3:
market_T3.head(3)

,id,id_num,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,vl_total_veiculos_leves,vl_total_veiculos_pesados,fl_optante_simples,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,vl_frota,empsetorcensitariofaixarendapopulacao,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_masculino,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,0,1,60,4,2,11,2,14,1,0,0,0,0,0,2709,0,1,1,1,32,10,0,0,0,0,0,1,8,0,0,0,20,5,0,23,0,0,0,0,0,7,49,1,2,2,0,44,47,41,2,2,6,8,6,6,0,0,0,0,220,536,57,57,536,220,0,0,0,0,0,0,0,6,0,0,0,2,1,5,0,0,1,5,0,21809443,3634907,13188096,210000,22019443,3669907,13188096,210000,0,0,0,0,0,6,21809443,2,2,3132172,3132172,119,26,100,0,1,4,6,5,3,4,2,0,1,1,25,0,26,19,0,0,119,145,13,93,0,43,93,19,26,26,27,0,-3,0,0
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,1,1,17,3,5,21,5,1,0,0,0,0,1,0,6811,0,1,1,1,3,3,0,0,0,0,0,2,0,0,0,2,17,2,1,9,1,1,0,0,468,4,70,1,1,1,0,27,27,27,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,10,210000,210000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,2,1,17,1,5,32,5,7,3,0,0,0,1,0,4783,0,0,1,1,87,21,0,0,0,0,0,1,0,0,0,1,2,0,0,26,3,1,0,0,713,2,44,1,1,1,0,32,32,32,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,50000,50000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
# Portfolio after T3
portfolio_T3 = pd.DataFrame(portfolio['id'])
portfolio_T3['Tr_exc'] = portfolio_T3['id'].isin(market_T3['id'])
portfolio_T3 = portfolio_T3[portfolio_T3['Tr_exc'] == True]
portfolio_T3 = portfolio_T3.drop(['Tr_exc'], axis = 1)
portfolio_T3 = pd.merge(portfolio_T3, market_T3, on='id', how='left')
# Full Portfolio View:
portfolio_T3.head(3)

,id,id_num,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,vl_total_veiculos_leves,vl_total_veiculos_pesados,fl_optante_simples,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,vl_frota,empsetorcensitariofaixarendapopulacao,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_masculino,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
0,09e95c1a84049001d086470a8f320a19b076f955a89122...,1646,1,1,0,0,0,5,11,1,0,0,0,0,0,3409,0,1,1,0,0,0,0,0,1,4,0,0,0,0,4,0,0,5,0,23,0,0,0,46132,2791,18,58,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,6,327487200,327487200,451,1361,81,18,0,23,411,508,282,103,22,10,2,2,523,836,1361,25,0,0,451,1812,84,357,-6,99,357,25,1361,1361,1383,0,-1,0,0
1,dc9d155f5bcd3172031e2ec9431786e8871ff4b0ff4728...,2696,1,33,4,5,0,5,43,5,0,0,0,0,0,2618,0,0,0,1,0,0,0,0,1,20,16,0,8,16,20,0,20,5,0,23,0,0,0,1309391,1007,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,4,176802910,176802910,1359,991,42,57,11,59,104,157,129,163,113,132,123,54,414,523,991,23,0,0,1361,2352,80,5014,-9,149,526,23,991,991,1306,0,-24,0,0
2,16843c9ffb920170477118933798510d8d5f00f5c56c12...,3681,1,64,3,2,22,5,16,2,0,0,0,0,0,2870,0,0,1,0,36,11,0,0,1,1,1,0,1,1,1,0,17,5,0,23,0,0,0,60101,688,10,11,1,2,2,0,70,70,70,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,27818640,27818640,172,119,52,47,1,10,27,30,20,15,11,3,2,19,9,91,119,4,21,7,172,291,42,149,0,65,149,4,119,104,113,14,5,6,0


***
## T4 - Label clients:

In [26]:
# DataFrame preparation Step 4:
market_T4 = market_T3.copy()
portfolio_T4 = portfolio_T3.copy()

In [27]:
# assign value 1 to customers:
portfolio_T4['cliente'] = 1
# assign value 1 to non-customers:
market_T4['cliente'] = 0
# assign value 1 to portfolio's customers in market:
market_T4.loc[market_T4['id'].isin(portfolio_T4['id']), ['cliente']] = 1

In [28]:
# client and non-clients count:
market_T4['cliente'].value_counts()

0    461732
1       566
Name: cliente, dtype: int64

***
## T5 - Train_Test_split data:

In [29]:
# No use.
# Data normalization:
#clients = market_T4['cliente']
#market_T4 = market_T4.drop(['cliente'],axis =1)
#market_T4[market_T4.columns[2:]] = preprocessing.scale(market_T4[market_T4.columns[2:]])
#market_T4['cliente'] = clients

In [30]:
# random value
random_vl=42

> ### Create dataset for train and test:

In [31]:
# DataFrame preparation Step 5:
market_T5 = market_T4.copy()
portfolio_T5 = portfolio_T4.copy()

In [32]:
# market without current customers:
market_T5_without_clients = pd.DataFrame(market_T5['id'])
market_T5_without_clients['Tr_exc'] = market_T5_without_clients['id'].isin(portfolio_T5['id'])
market_T5_without_clients = market_T5_without_clients[market_T5_without_clients['Tr_exc'] != True]
market_T5_without_clients = market_T5_without_clients.drop(['Tr_exc'], axis = 1)
market_T5_without_clients = pd.merge(market_T5_without_clients, market_T5, on='id', how='left')

In [33]:
# Number of customers:
num_clients = len(portfolio_T5)
# get non-customers for balance dataset: 
non_clients_sample = market_T5_without_clients.sample(n=num_clients, random_state = random_vl)

In [34]:
# Create dataset with customers and non-customers
frames = [portfolio_T5, non_clients_sample ]
dataset = pd.concat(frames)

In [35]:
# Market without dataset entries:
market_without_dataset = pd.DataFrame(market_T5['id'])
market_without_dataset['Tr_exc'] = market_without_dataset['id'].isin(dataset['id'])
market_without_dataset = market_without_dataset[market_without_dataset['Tr_exc'] != True]
market_without_dataset = market_without_dataset.drop(['Tr_exc'], axis = 1)
market_without_dataset = pd.merge(market_without_dataset, market_T5, on='id', how='left')

> ### Split dataset:

In [36]:
# Split
dataset_train, dataset_test = train_test_split(dataset, test_size=0.33, random_state = random_vl, stratify= dataset['cliente'])

***
## Organized Data:

> ***
- Original market  : market_T1
- Original portfolio : portfolio
- Original portolio with other datas : portolio_T1
>***
- Prepared dataset : dataset
- Prepared dataset for train : dataset_train
- Prepared dataset for test : dataset_test
>***
- Market without dataset data : market_without_dataset

***
## Models:

In [37]:
# Inputs for models:
X_train = dataset_train.copy()
X_test = dataset_test.copy()
X_train = X_train.drop(['id', 'id_num','cliente'], axis = 1)
X_test = X_test.drop(['id', 'id_num', 'cliente'], axis = 1)

y_train = dataset_train['cliente']
y_test = dataset_test['cliente']

> ### KNN Classifier:

In [38]:
# model:
knn_cl = KNeighborsClassifier(n_neighbors=5)
# Fit data to model:
knn_cl.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [39]:
# prediction:
pred = knn_cl.predict(X_test)
# confusion matrix:
print (confusion_matrix(y_test,pred))

[[177  10]
 [  0 187]]


In [40]:
# Classification report:
print (classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97       187
           1       0.95      1.00      0.97       187

   micro avg       0.97      0.97      0.97       374
   macro avg       0.97      0.97      0.97       374
weighted avg       0.97      0.97      0.97       374



> ### Decision Tree:

In [41]:
# Model:
dtree = DecisionTreeClassifier(random_state = random_vl)
# Fit data to model:
dtree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [42]:
# prediction:
pred = dtree.predict(X_test)
# confusion matrix:
print (confusion_matrix(y_test,pred))

[[184   3]
 [  0 187]]


In [43]:
# Classification report:
print (classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       187
           1       0.98      1.00      0.99       187

   micro avg       0.99      0.99      0.99       374
   macro avg       0.99      0.99      0.99       374
weighted avg       0.99      0.99      0.99       374



> ### SVM:

In [44]:
# Model:
svm_svc = SVC(gamma='auto', random_state = random_vl)
# Fit data to model:
svm_svc.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [45]:
# prediction:
pred = svm_svc.predict(X_test)
# confusion matrix:
print (confusion_matrix(y_test,pred))

[[  6 181]
 [  0 187]]


In [46]:
# Classification report:
print (classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       1.00      0.03      0.06       187
           1       0.51      1.00      0.67       187

   micro avg       0.52      0.52      0.52       374
   macro avg       0.75      0.52      0.37       374
weighted avg       0.75      0.52      0.37       374



> ### Random Forest

In [47]:
# Model:
rf = RandomForestClassifier(random_state = random_vl)
# Fit data to model:
rf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [48]:
# prediction:
pred = rf.predict(X_test)
# confusion matrix:
print (confusion_matrix(y_test,pred))

[[183   4]
 [  0 187]]


In [49]:
# Classification report:
print (classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99       187
           1       0.98      1.00      0.99       187

   micro avg       0.99      0.99      0.99       374
   macro avg       0.99      0.99      0.99       374
weighted avg       0.99      0.99      0.99       374



> ### Logistic Regression:

In [50]:
# Model:
lr = LogisticRegression(random_state = random_vl)
# Fit data to model:
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [51]:
# prediction:
pred = lr.predict(X_test)
# confusion matrix:
print (confusion_matrix(y_test,pred))

[[ 25 162]
 [  5 182]]


In [52]:
# Classification report:
print (classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.83      0.13      0.23       187
           1       0.53      0.97      0.69       187

   micro avg       0.55      0.55      0.55       374
   macro avg       0.68      0.55      0.46       374
weighted avg       0.68      0.55      0.46       374



***
## Benchmark:

In [53]:
# Comparative DataFrame:
performance = []
classifier = ['KNN', 'Decision Tree', 'SVM', 'Random Forest', 'Logistic Regression']
models = [knn_cl, dtree, svm_svc, rf, lr]
models_name = ['knn_cl', 'dtree', 'svm_svc', 'rf', 'lr']
for i in models:
    model = i
    model.fit(X_train,y_train)
    prediction = model.predict(X_test)    
    performance.append(metrics.accuracy_score(prediction,y_test))
        
models_dataframe=pd.DataFrame(performance,index = classifier)   
models_dataframe.columns = ['Accuracy']
models_dataframe['Model_name'] = models_name
models_dataframe['Model_description'] = models
models_dataframe

,Accuracy,Model_name,Model_description
KNN,0.973262,knn_cl,"KNeighborsClassifier(algorithm='auto', leaf_si..."
Decision Tree,0.991979,dtree,"DecisionTreeClassifier(class_weight=None, crit..."
SVM,0.516043,svm_svc,"SVC(C=1.0, cache_size=200, class_weight=None, ..."
Random Forest,0.989305,rf,"(DecisionTreeClassifier(class_weight=None, cri..."
Logistic Regression,0.553476,lr,"LogisticRegression(C=1.0, class_weight=None, d..."


In [54]:
# Best model name:
best_model_name = models_dataframe.Model_name[models_dataframe['Accuracy'] == models_dataframe['Accuracy'].max()].tolist()
print(best_model_name[0])

dtree


In [55]:
# Melhor modelo:
best_model = models_dataframe.Model_description[models_dataframe['Accuracy'] == models_dataframe['Accuracy'].max()].tolist()
best_model = best_model[0]

***
## Apply the best model:

In [56]:
# Fit train data:
best_model.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [57]:
# Adjust dataset to predict:
target_market = market_without_dataset.drop(['id', 'id_num','cliente'], axis = 1)

In [58]:
# prediction:
pred = best_model.predict(target_market)

In [59]:
# Count predictions:
market_without_dataset['cliente'] = pred
market_without_dataset['cliente'].value_counts()

0    458416
1      2750
Name: cliente, dtype: int64

***
## Results: 

> ### DataFrame with possible customers:

In [60]:
recommendation = market_without_dataset.copy()
recommendation = recommendation[recommendation['cliente'] == 1]

In [61]:
# All similar customers:
recommendation_full = pd.DataFrame()
recommendation_full['id'] = recommendation['id']
recommendation_full = pd.merge(recommendation_full, market_T1, on='id', how='left')

In [62]:
# save csv file:
# recommendation.to_csv('recommendation.csv', index=False)

# View all similar DataFrame
recommendation_full.head(3)

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,fl_optante_simples,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,vl_potenc_cons_oleo_gas,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,vl_frota,empsetorcensitariofaixarendapopulacao,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_st_suspensa,qt_socios_masculino,qt_socios_feminino,qt_socios_pep,qt_alteracao_socio_total,qt_alteracao_socio_90d,qt_alteracao_socio_180d,qt_alteracao_socio_365d,qt_socios_pj_ativos,qt_socios_pj_nulos,qt_socios_pj_baixados,qt_socios_pj_suspensos,qt_socios_pj_inaptos,vl_idade_media_socios_pj,vl_idade_maxima_socios_pj,vl_idade_minima_socios_pj,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,coligada_mais_nova_baixada,coligada_mais_antiga_baixada,idade_media_coligadas_baixadas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,qt_funcionarios_coligados,qt_funcionarios_coligados_gp,media_funcionarios_coligados_gp,max_funcionarios_coligados_gp,min_funcionarios_coligados_gp,vl_folha_coligados,media_vl_folha_coligados,max_vl_folha_coligados,min_vl_folha_coligados,vl_folha_coligados_gp,media_vl_folha_coligados_gp,max_vl_folha_coligados_gp,min_vl_folha_coligados_gp,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,total_filiais_coligados,media_filiais_coligados,max_filiais_coligados,min_filiais_coligados,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_ate_18,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,grau_instrucao_macro_desconhecido,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,id_num
0,8a584390e7b2fba5ef33a51b2f8fd63bd202194db3e5ee...,False,SOCIEDADE DE ECONOMIA MISTA,AC,ENTIDADES EMPRESARIAIS,SERVICOS FINANCEIROS,SERVIÇO,6.912329,5 a 10,False,True,False,False,False,2

In [63]:
print("From a total of %d companies, we found similarities with their portfolio in %d."%(market_without_dataset.shape[0], recommendation_full.shape[0]))

From a total of 461166 companies, we found similarities with their portfolio in 2750.


***

## Top25 Recommendation

In [64]:
# Prepare data
target_top25 = recommendation.drop(['id', 'id_num','cliente'], axis =1)

In [65]:
# Fit data:
nbrs = NearestNeighbors(n_neighbors=25, metric='euclidean').fit(target_top25)

In [66]:
# Prepare target customer only customers on portfolio:
target_customer = portfolio_T3.drop(['id', 'id_num'], axis =1)

# Target Customer Creation:
default_customer = target_customer.mode()
default_customer = pd.DataFrame(default_customer.iloc[0]).reset_index()
default_customer.columns = ['Columns', 'Sample_values']
default_customer = list(default_customer['Sample_values'])

In [67]:
# values default customer:
print(default_customer)

[1.0, 60.0, 1.0, 2.0, 0.0, 5.0, 43.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2901.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 5.0, 0.0, 23.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 44.0, 1.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 3.0, 11024424.0, 11024424.0, 52.0, 71.0, 32.0, 67.0, 0.0, 6.0, 16.0, 16.0, 16.0, 5.0, 3.0, 2.0, 0.0, 0.0, 38.0, 3.0, 71.0, 2.0, 0.0, 0.0, 124.0, 246.0, 29.0, 5014.0, 0.0, 47.0, 265.0, 2.0, 71.0, 69.0, 68.0, 0.0, 0.0, 0.0, 0.0]


In [68]:
# Top25 Recommendation:
print(nbrs.kneighbors([default_customer]))

(array([[ 72912.51308932,  73098.24151784,  73107.74548569,
        218621.90411301, 291459.52789195, 291486.92600527,
        292162.7862973 , 307629.03953463, 330065.50382613,
        332042.9619989 , 334553.65452047, 382330.62467451,
        395955.92585792, 437129.18292994, 437160.07549524,
        464311.81659958, 510710.67097722, 582863.45316549,
        582866.57604121, 585089.89207471, 585711.94832187,
        608255.91409948, 646965.56950119, 655692.07749675,
        655709.9882509 ]]), array([[1398,  939, 1544,  271,  278, 2416, 1112, 2044, 2657, 2548, 1463,
         395, 1689, 1766,  725, 1809, 2022, 1124,  632, 2280, 2518,  767,
         398, 1841,  499]]))


In [69]:
# Collection of recommended indexes:
distances, indices= nbrs.kneighbors([default_customer])
distances = distances[0].tolist()
indices = indices[0].tolist()
print(indices) # These indexes are from the training dataset (target_top25 and recommendation)

[1398, 939, 1544, 271, 278, 2416, 1112, 2044, 2657, 2548, 1463, 395, 1689, 1766, 725, 1809, 2022, 1124, 632, 2280, 2518, 767, 398, 1841, 499]


In [100]:
# Prepare data:
top25_recommendation = recommendation.iloc[indices]
top25_recommendation_ids = pd.DataFrame()
top25_recommendation_ids['id'] = top25_recommendation['id']
# Euclidean distance from target:
top25_recommendation_ids['distance'] = distances

top25_recommendation_ids

,id,distance
240819,a0050bd92917243c595badc478e23c48caab33eab29940...,72912.513089
160401,939cafd5935957cfd2ed4ff06f6d81e1ef7ccb4cba3b66...,73098.241518
262283,083d69f9ea8e6cd037ee83f1c15bca714ef3d910015dd0...,73107.745486
45697,3bc22585410624543970edbb68aa5b509f8077f112c551...,218621.904113
46141,66f097283634301e7ecdd5bd342941b44d40d91a83ba61...,291459.527892
401555,218585e197e353e387dbb0ffc24563f391bce973389561...,291486.926005
191608,8ff004e3a8994814c2d1f7e548712537445495fd15dae6...,292162.786297
341028,d3a6471c9129ffd4ae12e6ef30aee54008e277775dfaa7...,307629.039535
445026,bfa952631c59ccb3a1066823b91ab6fd486143f46a2d07...,330065.503826
425744,e176ca83a7f3ee31db973ff5f127fc7eeb3ac8bc261964...,332042.961999


In [71]:
# Full Dataset of Top25_recommendations:
full_top25_recommendation = pd.DataFrame()
full_top25_recommendation = top25_recommendation['id']
full_top25_recommendation = pd.merge(full_top25_recommendation, market_T1, on='id', how='left')
full_top25_recommendation.head()

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,fl_optante_simples,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,vl_potenc_cons_oleo_gas,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,vl_frota,empsetorcensitariofaixarendapopulacao,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_st_suspensa,qt_socios_masculino,qt_socios_feminino,qt_socios_pep,qt_alteracao_socio_total,qt_alteracao_socio_90d,qt_alteracao_socio_180d,qt_alteracao_socio_365d,qt_socios_pj_ativos,qt_socios_pj_nulos,qt_socios_pj_baixados,qt_socios_pj_suspensos,qt_socios_pj_inaptos,vl_idade_media_socios_pj,vl_idade_maxima_socios_pj,vl_idade_minima_socios_pj,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,coligada_mais_nova_baixada,coligada_mais_antiga_baixada,idade_media_coligadas_baixadas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,qt_funcionarios_coligados,qt_funcionarios_coligados_gp,media_funcionarios_coligados_gp,max_funcionarios_coligados_gp,min_funcionarios_coligados_gp,vl_folha_coligados,media_vl_folha_coligados,max_vl_folha_coligados,min_vl_folha_coligados,vl_folha_coligados_gp,media_vl_folha_coligados_gp,max_vl_folha_coligados_gp,min_vl_folha_coligados_gp,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,total_filiais_coligados,media_filiais_coligados,max_filiais_coligados,min_filiais_coligados,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_ate_18,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,grau_instrucao_macro_desconhecido,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais,id_num
0,a0050bd92917243c595badc478e23c48caab33eab29940...,True,CONDOMINIO EDILICIO,MA,ENTIDADES SEM FINS LUCRATIVOS,SERVICOS ADMINISTRATIVOS,SERVIÇO,6.008219,5 a 10,False,False,False,False,Fals

## Save recommendation in csv file:

In [72]:
# full_top25_recommendation.to_csv('full_top25_recommendation.csv', index=False)